<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.880 · TFM · Área 3 aula 1</p>
<p style="margin: 0; text-align:right;">2021-2 · Máster universitario en Ciencia de datos (<i>Data science</i>)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación

</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>


# TFM: 

## Clasificación de imágenes de recursión celular:

El coste de algunos medicamentos y tratamientos médicos ha subido tanto en los últimos años que muchos pacientes tienen que prescindir de ellos. Una de las razones más sorprendentes del coste es el tiempo que se tarda en sacar nuevos tratamientos al mercado. A pesar de las mejoras en la tecnología y la ciencia, la investigación y el desarrollo siguen retrasados. De hecho, encontrar nuevos tratamientos lleva, de media, más de 10 años y cuesta cientos de millones de dólares.

Recursion Pharmaceuticals, creadores del mayor conjunto de datos de imágenes biológicas del sector, generado íntegramente de forma interna, cree que la IA tiene el potencial de mejorar y agilizar drásticamente el proceso de descubrimiento de fármacos. Más concretamente, sus esfuerzos podrían ayudarles a entender cómo interactúan los fármacos con las células humanas.

En este proyecto se tiene que desentrañar el ruido experimental de las señales biológicas reales. La propuesta clasificará imágenes de células sometidas a una de las 1.108 perturbaciones genéticas diferentes. Puedes ayudar a eliminar el ruido introducido por la ejecución técnica y la variación ambiental entre experimentos.

Si se tiene éxito, se podría mejorar drásticamente la capacidad de la industria para modelar imágenes celulares según su biología relevante. A su vez, la aplicación de la IA podría disminuir en gran medida el coste de los tratamientos y garantizar que estos lleguen a los pacientes con mayor rapidez.


El proyecto que se presenta es el reto de la plataforma Kaggle alojado en https://www.kaggle.com/c/recursion-cellular-image-classification. Uno de los principales retos para aplicar la IA a los datos de microscopía biológica es que incluso las réplicas más cuidadosas de un proceso no parecerán idénticas. Este conjunto de datos supone un reto para desarrollar un modelo de identificación de réplicas que sea robusto frente al ruido experimental.

Los mismos siRNAs (perturbaciones genéticas efectivas) se han aplicado repetidamente a múltiples líneas celulares, para un total de 51 lotes experimentales. Cada lote tiene cuatro placas, cada una de las cuales tiene 308 pozos llenos. Para cada pozo, se ha realizado imágenes de microscopio desde dos perspectivas y a través de seis canales de imagen. No todos los lotes tienen necesariamente todos los pozos llenos o todos los siRNA presentes.

Hemos resumido esta descripción a lo esencial; para más detalles, consulte [RxRx.ai](https://www.rxrx.ai).


**El objetivo principal de la práctica es desarrollar modelos basados en el aprendizaje automático para clasificar con la mayor precisión posible, las perturbaciones genéticas aplicadas a las células.**

In [1]:
#%pylab inline
import sys
import os
import cv2
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import warnings
import numpy as np # linear algebra
import random
warnings.filterwarnings('ignore')

In [2]:
import skimage.io
from skimage.transform import resize
from imgaug import augmenters as iaa
from tqdm import tqdm
import PIL
from PIL import Image, ImageOps
from sklearn.utils import class_weight, shuffle
from keras.losses import binary_crossentropy, categorical_crossentropy
from keras.applications.densenet import preprocess_input
import keras.backend as K
import tensorflow as tf
from sklearn.metrics import f1_score, fbeta_score, cohen_kappa_score
from tensorflow.keras.utils import Sequence
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [3]:
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import (Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D, GlobalAveragePooling2D,
                          BatchNormalization, Input, Conv2D)
from keras.applications.densenet import DenseNet121
from keras.callbacks import ModelCheckpoint
from keras import metrics
from tensorflow.keras.optimizers import Adam, Nadam 
from keras import backend as K
import keras
from keras.models import Model

In [4]:
# All rellevant imports
import tensorflow as tf
import keras
import tensorflow_addons as tfa

In [5]:
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(tf.test.gpu_device_name())
tf.config.list_physical_devices('GPU')

### Variables constantes

In [6]:
SIZE = 224 # size of images
NUM_CLASSES = 1108

root_dir_train = '../input/recursion-cellular-image-classification-224-jpg/train/train/'
root_dir_test = '../input/recursion-cellular-image-classification-224-jpg/test/test/'

In [7]:
df_train = pd.read_csv('../input/recursion-cellular-image-classification-224-jpg/new_train.csv')
df_test = pd.read_csv('../input/recursion-cellular-image-classification-224-jpg/new_test.csv')

In [8]:
df_train['cell_type'] = df_train['experiment'].str.split('-').str[0]
df_test['cell_type'] = df_test['experiment'].str.split('-').str[0]

### Definición de imgaug

In [9]:
# https://github.com/aleju/imgaug
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
seq = iaa.Sequential([
    sometimes(
        iaa.OneOf([
            iaa.Add((-10, 10), per_channel=0.5),
            iaa.Multiply((0.9, 1.1), per_channel=0.5),
            iaa.ContrastNormalization((0.9, 1.1), per_channel=0.5)
        ])
    ),
    iaa.Fliplr(0.5),
    iaa.Crop(percent=(0, 0.1)),
],random_order=True)

### Definición del generador customizado

In [10]:
class My_Generator(Sequence):

    def __init__(self, image_filenames, labels, batch_size, is_train=True, augment=False):
        
        self.image_filenames, self.labels = image_filenames, labels
        self.batch_size = batch_size
        self.is_train = is_train
        self.is_augment = augment
        if(self.is_train):
            self.on_epoch_end()
    

    def __len__(self):
        return int(np.ceil(len(self.image_filenames) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.image_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]

        if(self.is_train):
            return self.train_generate(batch_x, batch_y)
        return self.valid_generate(batch_x, batch_y)

    def on_epoch_end(self):
        
        if(self.is_train):
            self.image_filenames, self.labels = shuffle(self.image_filenames, self.labels)
        else:
            pass

    def train_generate(self, batch_x, batch_y):
        
        batch_images = []
        
        for (sample, label) in zip(batch_x, batch_y):

            img = cv2.imread(root_dir_train + sample)

            if(self.is_augment):
                img = seq.augment_image(img)
            batch_images.append(img)
            
        batch_images = np.array(batch_images, np.float32)/255
        batch_y = np.array(batch_y, np.float32)
        
        return batch_images, batch_y

    def valid_generate(self, batch_x, batch_y):
        
        batch_images = []
        
        for (sample, label) in zip(batch_x, batch_y):
            img = cv2.imread(root_dir_train + sample)
            batch_images.append(img)
            
        batch_images = np.array(batch_images, np.float32)/255
        
        batch_y = np.array(batch_y, np.float32)
        return batch_images, batch_y

### Función para crear el modelo

In [11]:
def create_model(input_shape, n_out, weight_imagenet=True):
    
    input_tensor = tf.keras.Input(shape=input_shape)
    
    base_model = ''
    if weight_imagenet:
        base_model = tf.keras.applications.DenseNet121(include_top=False, weights='imagenet', input_tensor=input_tensor)
    else:
        base_model = tf.keras.applications.DenseNet121(include_top=False, weights=None, input_tensor=input_tensor)
    
    x = GlobalAveragePooling2D()(base_model.output)
    
    x = Dense(1024, activation='relu')(x)
    
    final_output = Dense(n_out, activation='softmax', name='final_output')(x)

    model = Model(input_tensor, final_output)

    return model

### División del conjunto de entrenamiento en train y validation

In [12]:
x = df_train['filename']
y = df_train['sirna']

x, y = shuffle(x, y, random_state=10)

y = to_categorical(y, num_classes=NUM_CLASSES)

train_x, valid_x, train_y, valid_y = train_test_split(x, y, test_size=0.15, stratify=y, random_state=10)

### Model DenseNet121 (transfer learning)

In [13]:
epochs = 10; batch_size = 128
train_generator = My_Generator(train_x, train_y, batch_size, is_train=True, augment=True)

In [14]:
model = create_model(input_shape=(SIZE,SIZE,3),n_out=NUM_CLASSES, weight_imagenet=True)

In [15]:
# unfreeze the last 5 layers
for layer in model.layers:
    layer.trainable = False

for i in range(-5,0):
    model.layers[i].trainable = True

model.compile(
    loss='categorical_crossentropy',
    optimizer=Nadam(1e-3))

model.fit(
    train_generator,
    steps_per_epoch=np.ceil(float(len(train_y)) / float(128)),
    epochs=15,
    verbose=1)

In [16]:
model.save_weights("./model_densenet121_freeze_layers_weight.h5")

### Model DenseNet121 (train from scratch)

In [17]:
epochs = 20; batch_size = 8
es = EarlyStopping(monitor='val_loss', verbose=1, patience=5, restore_best_weights=True)

In [18]:
train_generator = My_Generator(train_x, train_y, batch_size, is_train=True, augment=True)
valid_generator = My_Generator(valid_x, valid_y, batch_size, is_train=False)

In [19]:
model = create_model(input_shape=(SIZE,SIZE,3),n_out=NUM_CLASSES, weight_imagenet=False)

In [20]:
# train all layers
for layer in model.layers:
    layer.trainable = True

model.compile(optimizer=Nadam(1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.load_weights('../input/model-freeze-layers-weights/model_densenet121_freeze_layers_weight.h5')

history = model.fit(
            train_generator,
            steps_per_epoch=np.ceil(float(len(train_x)) / float(batch_size)),
            validation_data=valid_generator,
            validation_steps=np.ceil(float(len(valid_x)) / float(batch_size)),
            epochs=epochs,
            verbose=1,
            callbacks=[es])

In [21]:
model.save_weights("./model_dense121_trained_weight.h5")

## Models (HEPG2, HUVEC, RPE, U2OS)

In [22]:
df_train_hepg2 = df_train[df_train['cell_type'] == 'HEPG2']
df_train_huvec = df_train[df_train['cell_type'] == 'HUVEC']
df_train_rpe = df_train[df_train['cell_type'] == 'RPE']
df_train_u2os = df_train[df_train['cell_type'] == 'U2OS']

In [23]:
print('HEPG2 = ', len(df_train_hepg2))
print('HUVEC = ', len(df_train_huvec))
print('RPE = ', len(df_train_rpe))
print('U2OS = ', len(df_train_u2os))

In [24]:
epochs = 15; batch_size = 8

es = EarlyStopping(monitor='val_loss', verbose=1, patience=5, restore_best_weights=True)

#### HEPG2

In [25]:
x = df_train_hepg2['filename']
y = df_train_hepg2['sirna']

x, y = shuffle(x, y, random_state=10)
y = to_categorical(y, num_classes=NUM_CLASSES)
train_x, valid_x, train_y, valid_y = train_test_split(x, y, test_size=0.2, stratify=y, random_state=10)

# TRAIN, VALID GENERATORS
train_generator = My_Generator(train_x, train_y, batch_size, is_train=True, augment=False)
valid_generator = My_Generator(valid_x, valid_y, batch_size, is_train=False)

# CREATE MODEL
model_hepg2 = create_model(input_shape=(SIZE,SIZE,3),n_out=NUM_CLASSES, weight_imagenet=False)
model_hepg2.compile(optimizer=Nadam(5e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model_hepg2.load_weights('../input/model-trained-weights/model_dense121_trained_weight.h5')

# FIT
# train all layers
for layer in model_hepg2.layers:
    layer.trainable = True
history = model_hepg2.fit(
            train_generator,
            steps_per_epoch=np.ceil(float(len(train_x)) / float(batch_size)),
            validation_data=valid_generator,
            validation_steps=np.ceil(float(len(valid_x)) / float(batch_size)),
            epochs=epochs,
            verbose=1,
            callbacks=[es])

# SAVE WEIGHTS
model_hepg2.save_weights("./model_trained_hepg2_weight.h5")

#### HUVEC

In [26]:
x = df_train_huvec['filename']
y = df_train_huvec['sirna']

x, y = shuffle(x, y, random_state=10)
y = to_categorical(y, num_classes=NUM_CLASSES)
train_x, valid_x, train_y, valid_y = train_test_split(x, y, test_size=0.2, stratify=y, random_state=10)

# TRAIN, VALID GENERATORS
train_generator = My_Generator(train_x, train_y, batch_size, is_train=True, augment=False)
valid_generator = My_Generator(valid_x, valid_y, batch_size, is_train=False)

# CREATE MODEL
model_huvec = create_model(input_shape=(SIZE,SIZE,3),n_out=NUM_CLASSES, weight_imagenet=False)
model_huvec.compile(optimizer=Nadam(5e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model_huvec.load_weights('../input/model-trained-weights/model_dense121_trained_weight.h5')

# FIT
# train all layers
for layer in model_huvec.layers:
    layer.trainable = True
history = model_huvec.fit(
            train_generator,
            steps_per_epoch=np.ceil(float(len(train_x)) / float(batch_size)),
            validation_data=valid_generator,
            validation_steps=np.ceil(float(len(valid_x)) / float(batch_size)),
            epochs=epochs,
            verbose=1,
            callbacks=[es])

# SAVE WEIGHTS
model_huvec.save_weights("./model_trained_huvec_weight.h5")

#### RPE

In [27]:
x = df_train_rpe['filename']
y = df_train_rpe['sirna']

x, y = shuffle(x, y, random_state=10)
y = to_categorical(y, num_classes=NUM_CLASSES)
train_x, valid_x, train_y, valid_y = train_test_split(x, y, test_size=0.2, stratify=y, random_state=10)

# TRAIN, VALID GENERATORS
train_generator = My_Generator(train_x, train_y, batch_size, is_train=True, augment=False)
valid_generator = My_Generator(valid_x, valid_y, batch_size, is_train=False)

# CREATE MODEL
model_rpe = create_model(input_shape=(SIZE,SIZE,3),n_out=NUM_CLASSES, weight_imagenet=False)
model_rpe.compile(optimizer=Nadam(5e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model_rpe.load_weights('../input/model-trained-weights/model_dense121_trained_weight.h5')

# FIT
# train all layers
for layer in model_rpe.layers:
    layer.trainable = True
history = model_rpe.fit(
            train_generator,
            steps_per_epoch=np.ceil(float(len(train_x)) / float(batch_size)),
            validation_data=valid_generator,
            validation_steps=np.ceil(float(len(valid_x)) / float(batch_size)),
            epochs=epochs,
            verbose=1,
            callbacks=[es])

# SAVE WEIGHTS
model_rpe.save_weights("./model_trained_rpe_weight.h5")

#### U2OS

In [28]:
x = df_train_u2os['filename']
y = df_train_u2os['sirna']

x, y = shuffle(x, y, random_state=10)
y = to_categorical(y, num_classes=NUM_CLASSES)
train_x, valid_x, train_y, valid_y = train_test_split(x, y, test_size=0.2, stratify=y, random_state=10)

# TRAIN, VALID GENERATORS
train_generator = My_Generator(train_x, train_y, batch_size, is_train=True, augment=False)
valid_generator = My_Generator(valid_x, valid_y, batch_size, is_train=False)

# CREATE MODEL
model_u2os = create_model(input_shape=(SIZE,SIZE,3),n_out=NUM_CLASSES, weight_imagenet=False)
model_u2os.compile(optimizer=Nadam(5e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model_u2os.load_weights('../input/model-trained-weights/model_dense121_trained_weight.h5')

# FIT
# train all layers
for layer in model_u2os.layers:
    layer.trainable = True
history = model_u2os.fit(
            train_generator,
            steps_per_epoch=np.ceil(float(len(train_x)) / float(batch_size)),
            validation_data=valid_generator,
            validation_steps=np.ceil(float(len(valid_x)) / float(batch_size)),
            epochs=epochs,
            verbose=1,
            callbacks=[es])

# SAVE WEIGHTS
model_u2os.save_weights("./model_trained_u2os_weight.h5")

## Prediccion según el tipo de célula

In [29]:
model_hepg2 = create_model(input_shape=(SIZE,SIZE,3),n_out=NUM_CLASSES, weight_imagenet=False)

model_hepg2.compile(optimizer=Nadam(5e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_hepg2.load_weights('../input/model-hepg2/model_trained_hepg2_weight.h5')

In [30]:
model_huvec = create_model(input_shape=(SIZE,SIZE,3),n_out=NUM_CLASSES, weight_imagenet=False)

model_huvec.compile(optimizer=Nadam(5e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_huvec.load_weights('../input/model-huvec/model_trained_huvec_weight.h5')

In [31]:
model_rpe = create_model(input_shape=(SIZE,SIZE,3),n_out=NUM_CLASSES, weight_imagenet=False)

model_rpe.compile(optimizer=Nadam(5e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_rpe.load_weights('../input/model-rpe/model_trained_rpe_weight.h5')

In [32]:
model_u2os = create_model(input_shape=(SIZE,SIZE,3),n_out=NUM_CLASSES, weight_imagenet=False)

model_u2os.compile(optimizer=Nadam(5e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_u2os.load_weights('../input/model-u2os/model_trained_u2os_weight.h5')

In [33]:
def get_predict(img, name):
    
    score_predict = 0
    
    if 'HEPG2' in name:

        score_predict = model_hepg2.predict(img)

    elif 'HUVEC' in name:

        score_predict = model_huvec.predict(img)

    elif 'RPE' in name:

        score_predict = model_rpe.predict(img)

    else: ## U2OS

        score_predict = model_u2os.predict(img)
    
    return score_predict

### Predict utilizando ambos sites

In [34]:
id_code = df_test['id_code'].unique()

In [35]:
test_predict = []

for i, name in tqdm(enumerate(id_code)):
    
    #Predict S1
    image1 = cv2.imread(root_dir_test + name + '_s1.jpeg')
    image1 = (image1[np.newaxis])/255
    score_predict1 = get_predict(image1, name)
    
    #Predict S2
    image2 = cv2.imread(root_dir_test + name + '_s2.jpeg') 
    image2 = (image2[np.newaxis])/255
    score_predict2 = get_predict(image2, name)
    
    #Predict AVG
    score_avg = 0.5 * (score_predict1 + score_predict2)
    
    sirna = np.argmax(score_avg)
    
    test_predict.append([name, sirna])

In [36]:
df_submit = pd.DataFrame(test_predict, columns = ['id_code','sirna'])

In [37]:
df_submit

In [38]:
df_submit.to_csv('submit_v2.csv', index=False)

## Actualización de la predicción utilizando "plates leak"

In [39]:
train_csv = pd.read_csv("../input/recursion-cellular-image-classification-224-jpg/new_train.csv")
test_csv = pd.read_csv("../input/recursion-cellular-image-classification-224-jpg/new_test.csv")
sub = pd.read_csv("../input/submitv2/submit_v2.csv")

In [40]:
np.stack([train_csv.plate.values[train_csv.sirna == i] for i in range(10)]).transpose()

In [41]:
# you will see the same output here for each sirna number
train_csv.loc[train_csv.sirna==0,'plate'].value_counts() / 2

In [42]:
plate_groups = np.zeros((1108,4), int)
for sirna in range(1108):
    grp = train_csv.loc[train_csv.sirna==sirna,:].plate.value_counts().index.values
    assert len(grp) == 3
    plate_groups[sirna,0:3] = grp
    plate_groups[sirna,3] = 10 - grp.sum()
    
plate_groups[:10,:]

In [43]:
new_test = test_csv.drop(test_csv[test_csv.filename.str.contains('_s2.jpeg')].index)

In [44]:
all_test_exp = test_csv.experiment.unique()

group_plate_probs = np.zeros((len(all_test_exp),4))
for idx in range(len(all_test_exp)):
    
    preds = sub.loc[test_csv.experiment == all_test_exp[idx],'sirna'].values
    
    pp_mult = np.zeros((len(preds),1108))
    
    pp_mult[range(len(preds)),preds] = 1
    
    sub_test = new_test.loc[test_csv.experiment == all_test_exp[idx],:]
    
    assert len(pp_mult) == len(sub_test)
        
    for j in range(4):
        mask = np.repeat(plate_groups[np.newaxis, :, j], len(pp_mult), axis=0) == \
               np.repeat(sub_test.plate.values[:, np.newaxis], 1108, axis=1)
        
        group_plate_probs[idx,j] = np.array(pp_mult)[mask].sum()/len(pp_mult)

In [45]:
pd.DataFrame(group_plate_probs, index = all_test_exp)

In [46]:
exp_to_group = group_plate_probs.argmax(1)
print(exp_to_group)

In [47]:
predicted = []

for i, name in tqdm(enumerate(id_code)):    
    image1 = cv2.imread(root_dir_test + name + '_s1.jpeg')
    image1 = (image1[np.newaxis])/255
    score_predict1 = get_predict(image1, name)
    
    image2 = cv2.imread(root_dir_test + name + '_s2.jpeg') 
    image2 = (image2[np.newaxis])/255
    score_predict2 = get_predict(image2, name)
    
    predicted.append(0.5 * (score_predict1 + score_predict2))

In [48]:
predicted = np.stack(predicted).squeeze()

In [49]:
def select_plate_group(pp_mult, idx):
    
    sub_test = new_test.loc[test_csv.experiment == all_test_exp[idx],:]
    
    assert len(pp_mult) == len(sub_test)
    
    mask = np.repeat(plate_groups[np.newaxis, :, exp_to_group[idx]], len(pp_mult), axis=0) != \
           np.repeat(sub_test.plate.values[:, np.newaxis], 1108, axis=1)
    
    pp_mult[mask] = 0
    
    return pp_mult

In [50]:
sub_copia = sub.copy()

In [51]:
indices = (test_csv.experiment == all_test_exp[idx])

In [52]:
for idx in range(len(all_test_exp)):

    indices = (new_test.experiment == all_test_exp[idx])
    
    preds = predicted[indices,:].copy()
    
    preds = select_plate_group(preds, idx)
    sub_copia.loc[indices,'sirna'] = preds.argmax(1)

In [53]:
sub_copia.to_csv('submit_v3.csv', index=False)